<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
#создаем окружение

import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
#создаем спарк-сессию
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "natasha pritykovskaya Spark Dataframe app") 

spark = SparkSession.builder.config(conf=conf).appName("natasha pritykovskaya Spark Dataframe app").getOrCreate()

In [3]:
spark

In [64]:
sc = spark.sparkContext

In [65]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [66]:
# предопределяем названия и тип данных для будущей таблицы
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("rating", IntegerType()),
    StructField("timestamp", IntegerType())
])

In [67]:
#читаем файл (предварительно записанный в личную папку на hdfs)
%%time
df = spark.read\
          .schema(schema)\
          .format("csv")\
          .option("sep", "\t")\
          .load('u.data')

CPU times: user 3.97 ms, sys: 0 ns, total: 3.97 ms
Wall time: 28.4 ms


In [68]:
df.show(5, vertical=True, truncate=False)

-RECORD 0--------------
 user_id   | 196       
 item_id   | 242       
 rating    | 3         
 timestamp | 881250949 
-RECORD 1--------------
 user_id   | 186       
 item_id   | 302       
 rating    | 3         
 timestamp | 891717742 
-RECORD 2--------------
 user_id   | 22        
 item_id   | 377       
 rating    | 1         
 timestamp | 878887116 
-RECORD 3--------------
 user_id   | 244       
 item_id   | 51        
 rating    | 2         
 timestamp | 880606923 
-RECORD 4--------------
 user_id   | 166       
 item_id   | 346       
 rating    | 1         
 timestamp | 886397596 
only showing top 5 rows



In [69]:
#преобразуем в нужный формат, чтобы можно было писать к нему sql запросы
df.registerTempTable("df")

In [70]:
query = """
SELECT rating, count(*) FROM df 
WHERE item_id == 79 group by rating order by 1 asc
"""

**Придраться не к чему :) Только можно было вместо "a", "b", "query1" и "query2" назавть переменные с большим смыслом :)**

In [73]:
# пишем статистику по фильму в pandas датафрейм (так удобнее)
a = spark.sql(query).toPandas()

In [74]:
query2 = """
SELECT rating, count(*) FROM df 
group by rating order by 1 asc
"""

In [75]:
# пишем статистику по всем фильмам в pandas датафрейм 
b = spark.sql(query2).toPandas()

In [79]:
dict_res = {
   "hist_film": list(a['count(1)']),
   "hist_all": list(b['count(1)'])
}

In [81]:
import json

with open('lab06.json', 'wt') as file:
    file.writelines(json.dumps(dict_res))

In [82]:
#останавливаем спарк контекст
spark.stop()